In [14]:
import pandas as pd
import numpy as np
import sqlite3

In [27]:
#Load data
google_df = pd.read_csv('/Users/marianaarrieta/Documents/Data_Projects/london_restaurants/raw_data/data_google_maps.csv')
opentable_df = pd.read_csv('/Users/marianaarrieta/Documents/Data_Projects/london_restaurants/raw_data/opentable_data.csv')


In [9]:
google_df.head()

,names,summary,cusine,price,reviews_count,rating,address,url
0,Restaurant Story,Chef Tom Sellers' novel approach to British cl...,Fine dining restaurant,££££,418 reviews,4.6,"199 Tooley St, London SE1 2JX",https://www.google.com/maps/place/Restaurant+S...
1,Galvin La Chapelle,NaN,French restaurant,££££,"1,901 reviews",4.6,NaN,https://www.google.com/maps/place/Galvin+La+Ch...
2,City Social,NaN,Modern European restaurant,££££,"1,187 reviews",4.5,NaN,https://www.google.com/maps/place/City+Social/...
3,Brawn,Mediterranean small plates menu with an 'all t...,Mediterranean restaurant,££,615 reviews,4.7,"49 Columbia Rd, London E2 7RG",https://www.google.com/maps/place/Brawn/@51.52...
4,Il Bordello,Old-school Italian restaurant with an authenti...,Italian restaurant,££,"1,170 reviews",4.5,"metropolitan wharf, 70 Wapping Wall, London E1...",https://www.google.com/maps/place/Il+Bordello/...


In [10]:
opentable_df.head()

,names,cuisine,price,reviews,rating,bookings,address
0,Seasons Mayfair,Fish,4,41.0,4.3,8.0,Mayfair
1,PAPA L's KITCHEN,African,4,257.0,4.6,6.0,St. James's
2,Coqbull Soho,Rotisserie Chicken,2,175.0,4.1,47.0,Soho
3,Ham Yard Bar and Restaurant,Modern European,2,3696.0,4.6,92.0,Soho
4,El Norte,Spanish,4,296.0,4.5,45.0,Mayfair


## Understand the Differences in Data

In [28]:
#Different values in price
print(google_df.price.unique())
print(opentable_df.price.unique())

#Price ranges from 1 to 4, but we need to have the same format across tables
price_dict = {'£':1, '££':2, '£££':3, '££££':4, 'Not Available': np.nan}
#Map values
google_df['price'] = google_df['price'].map(price_dict)


['££££' '££' 'Not Available' '£££' '£']
[4 2 3]


0      4.0
1      4.0
2      4.0
3      2.0
4      2.0
      ... 
373    NaN
374    NaN
375    NaN
376    NaN
377    NaN
Name: price, Length: 378, dtype: float64

In [42]:
#Reviews have also different formats, so we have to make them all integers
google_df['reviews_count'] = google_df['reviews_count'].replace(',','', regex=True)
google_df['reviews_count'] = google_df['reviews_count'].replace('[a-zA-Z]','', regex=True)
google_df['reviews_count'] = google_df['reviews_count'].replace('',np.nan, regex=True)
google_df['reviews_count'] = google_df['reviews_count'].replace(""" ''""",np.nan, regex=True)
google_df['reviews_count'] = google_df['reviews_count'].str.strip()
google_df['reviews_count'] = google_df['reviews_count'].astype(float)

In [48]:
#Make sure caps are the same in restaurant titles to be able to merge later
google_df['names'] = google_df['names'].str.lower()
google_df['names'] = google_df['names'].str.title()
opentable_df['names'] = opentable_df['names'].str.lower()
opentable_df['names'] = opentable_df['names'].str.title()

,names,summary,cusine,price,reviews_count,rating,address,url
0,Restaurant Story,Chef Tom Sellers' novel approach to British cl...,Fine dining restaurant,4.0,418.0,4.6,"199 Tooley St, London SE1 2JX",https://www.google.com/maps/place/Restaurant+S...
1,Galvin La Chapelle,NaN,French restaurant,4.0,1901.0,4.6,NaN,https://www.google.com/maps/place/Galvin+La+Ch...
2,City Social,NaN,Modern European restaurant,4.0,1187.0,4.5,NaN,https://www.google.com/maps/place/City+Social/...
3,Brawn,Mediterranean small plates menu with an 'all t...,Mediterranean restaurant,2.0,615.0,4.7,"49 Columbia Rd, London E2 7RG",https://www.google.com/maps/place/Brawn/@51.52...
4,Il Bordello,Old-school Italian restaurant with an authenti...,Italian restaurant,2.0,1170.0,4.5,"metropolitan wharf, 70 Wapping Wall, London E1...",https://www.google.com/maps/place/Il+Bordello/...


### Clean Google Data

In [12]:
#Remove duplicates, if any 
google_df.drop_duplicates()

#Remove rows where cusine = "Not a restaurant"
google_df = google_df[google_df.cusine != 'Not a restaurant']

#Remove rows where there are no ratings 
google_df = google_df[google_df.rating != 'Not Available']


In [13]:
len(google_df)

373